In [ ]:
# pip install cobrakbase --upgrade
# !pip list

In [23]:
## Import necessary modules to rune ModelSEEDpy & CobraPy

import cobra
import cplex
import optlang
import sys
import json
import logging
import pandas as pd

from configparser import ConfigParser
config = ConfigParser()
config.read("config.cfg")
paths = config.get("script", "syspaths").split(";")
for path in paths:
    sys.path.append(path)
import cobrakbase
from cobrakbase.core.kbasefba.newmodeltemplate_builder import NewModelTemplateBuilder
from cobra.core import Gene, Metabolite, Model, Reaction
from modelseedpy import MSCommunity,MSATPCorrection,MSPackageManager, FBAHelper, MSBuilder, MSGapfill
from modelseedpy.helpers import get_template
from IPython.display import HTML


In [7]:
kbase_api = cobrakbase.KBaseAPI("JUJ3IEQBT52UBBGKML4XFVZLYBWO5XFT")

#model = kbase_api.get_from_ws("Cjaponicus_0Yield_Model",109126)
#model = kbase_api.get_from_ws("Ecoli_gF44_Model",109126)
model = kbase_api.get_from_ws("Cj_gF7c",161465)

media1 = kbase_api.get_from_ws("ChitinM9MinMedia",162658)               # Chitin Min Media
media2 = kbase_api.get_from_ws("NAGM9MinMedia",162658)                  # NAG Min Media
media3 = kbase_api.get_from_ws("GlucoseM9MinMedia",162658)              # Glucose Min Media

model.medium = media1
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


In [ ]:
solution = model.optimize()
pd.set_option('display.max_rows', None)
print(solution.fluxes)

In [ ]:
#model = kbase_api.get_from_ws("Cjaponicus_0Yield_wMets_Model",109126)
model = kbase_api.get_from_ws("Ecoli_wMets_Model",109126)

media1 = kbase_api.get_from_ws("ChitinM9Media",109126)               # Chitin Min Media
media2 = kbase_api.get_from_ws("NAGM9Media",109126)                  # NAG Min Media
media3 = kbase_api.get_from_ws("Cjaponicus_Mets_Media",109126)
media4 = kbase_api.get_from_ws("Ecoli_Mets_Media",109126)

model.medium = media4

model.summary()

In [ ]:
solution = model.optimize()
solution.fluxes

In [266]:
## Pull the community model and media (2) out of KBase

#Token for KBase (needs to be updated monthly)
kbase_api = cobrakbase.KBaseAPI("R3DCUWDTINXXDKMMTF6MJX3VWR7KUAQ4")

model = kbase_api.get_from_ws("Comm10_Model",169863)
media1 = kbase_api.get_from_ws("ChitinM9MinMedia",161465)               # Chitin Min Media
media2 = kbase_api.get_from_ws("NAGM9MinMedia",161465)                  # NAG Min Media
media3 = kbase_api.get_from_ws("Chitin_Comm_Media",109126)
media4 = kbase_api.get_from_ws("NAG_Comm_Media",109126)

In [267]:
SK = Reaction('SK_cpd03091_c0','','',0,1000)
SK.add_metabolites({model.metabolites.cpd03091_c1:-1})
model.add_reactions([SK])

In [268]:
model.reactions.get_by_id('ChitinHydrolysis_c1').add_metabolites({
    'cpd00001_c1': 1.925,
    'cpd00002_c1': 1.925,
    'cpd00008_c1': -1.925,
    'cpd00009_c1': -1.925,
    'cpd00067_c1': -1.925
})

model.reactions.get_by_id('ChitinHydrolysis_c1').add_metabolites({
    'cpd00001_c1': -25,
    'cpd00002_c1': -25,
    'cpd00008_c1': 25,
    'cpd00009_c1': 25,
    'cpd00067_c1': 25
})
model.reactions.get_by_id('ChitinHydrolysis_c1')

Reaction identifier,ChitinHydrolysis_c1
Name,CustomReaction_c1
Memory address,0x019ed5ef70
Stoichiometry,25 cpd00001_c1 + cpd00001_e0 + 25 cpd00002_c1 + cpd11683_e0 --> 25 cpd00008_c1 + 25 cpd00009_c1 + 25 cpd00067_c1 + cpd00122_c1 25 H2O_c0 + H2O_e0 + 25 ATP_c0 + Chitin --> 25 ADP_c0 + 25 Phosphate_c0 + 25 H+_c0 + N-Acetyl-D-glucosamine_c0
GPR,
Lower bound,0
Upper bound,1000


In [269]:
## Create a community model analysis object in ModelSEEDpy

model.solver = 'optlang-cplex'

# Reactions to change bounds -- (0, inf) makes => , (-inf,0) makes <=
    # Reactions to limit amount of NAG taken up by individual isolates. Otherwise, all NAG goes to E. coli.
#model.reactions.get_by_id("rxn05485_c2").bounds = 0,5  #NAG transport e0 <=> c2
#model.reactions.get_by_id("rxn05485_c1").bounds = 0,5  #NAG transport e0 <=> c1
    # ^ The second reaction isn't necessary if community is grown on chitin. Only use if running on NAG.
    
# Community model is 95% C. japonicus and 5% E. coli -- included in code below
#comm = MSCommunity(model,["Cjaponicus","Ecoli"],{"Cjaponicus":0.5,"Ecoli":0.5})
#comm = MSCommunity(model,["Cjaponicus","Ecoli"],{"Cjaponicus":0.95124,"Ecoli":0.04876})
comm = MSCommunity(model,["Cjaponicus","Ecoli"],{"Cjaponicus":0.95,"Ecoli":0.05})
comm.lp_filename = "Cjaponicus_Ecoli"

#print("exchanges", comm.model.exchanges) #All exchange rxns from community model.

In [22]:
model.medium = media1
cobra.io.save_matlab_model(model, "Comm0_Model.mat", "model")

In [6]:
## Now we use a utility to test that each compartment in the model can grow and make ATP
    # This helps us ensure that individual isolates are showing correct growth & not generating excessive ATP  

output = comm.test_individual_species(media1,allow_interaction=False) # Test on Chitin - No growth by E. coli
output

,Species,Biomass,ATP
0,Cjaponicus,<Solution 1.156 at 0x1527a0430>,<Solution 192.500 at 0x1527a08b0>
1,Ecoli,<Solution 0.000 at 0x1527a0eb0>,<Solution 0.000 at 0x15255f310>


In [7]:
## Repeat utility but test on NAG -- Should see growth by both isolates

output = comm.test_individual_species(media2,allow_interaction=False)
output

,Species,Biomass,ATP
0,Cjaponicus,<Solution 1.156 at 0x152cdafa0>,<Solution 192.500 at 0x1526f3280>
1,Ecoli,<Solution 0.000 at 0x1526f3580>,<Solution 262.500 at 0x1526f3970>


In [8]:
## Repeat utility but test on NAG (with interactions)

output = comm.test_individual_species(media2,allow_interaction=True)
output

,Species,Biomass,ATP
0,Cjaponicus,<Solution 1.437 at 0x1522274c0>,<Solution 318.750 at 0x1526a4400>
1,Ecoli,<Solution 1.438 at 0x1526a4700>,<Solution 318.750 at 0x1526a4af0>


In [ ]:
## !!!!!!!!!! ONLY RUN IF NECESSARY! !!!!!!!!!!

## Code to remove reactions that are creating ATP issue.
    # Note - the modified community model cannot yet be saved directly to KBase
    #S o we use the model editor in KBase to manually remove problem reaction

#Note the large amount of ATP generated by Cjaponicus - let's use the ATP correction tool to fix this
core = NewModelTemplateBuilder.from_dict(get_template('template_core'), None).build()

#We need to use one of the media in narrative https://narrative.kbase.us/narrative/94026
#These are the medias that work with the ATP correction tool
glucosemedia = kbase_api.get_from_ws("Glc.O2.atp",94026)

#Initialize the ATP correction object selecting the c1 compartment, which corresponds to Cjaponicus
msatp = MSATPCorrection(comm.model,core,[glucosemedia],"c2")
msatp.evaluate_growth_media()
msatp.determine_growth_media()
msatp.apply_growth_media_gapfilling()
msatp.expand_model_to_genome_scale()

In [9]:
## If reactions have been removed, we can repeat utility test.

output = comm.test_individual_species(media1,allow_interaction=True) # Test on chitin and allow interactions
output

,Species,Biomass,ATP
0,Cjaponicus,<Solution 1.437 at 0x1526fe130>,<Solution 318.750 at 0x1526fe580>
1,Ecoli,<Solution 1.438 at 0x1526fe850>,<Solution 318.750 at 0x1526fec40>


In [270]:
model.reactions.get_by_id('ChitinHydrolysis_c1')

Reaction identifier,ChitinHydrolysis_c1
Name,CustomReaction_c1
Memory address,0x019ed5ef70
Stoichiometry,25 cpd00001_c1 + cpd00001_e0 + 25 cpd00002_c1 + cpd11683_e0 --> 25 cpd00008_c1 + 25 cpd00009_c1 + 25 cpd00067_c1 + cpd00122_c1 25 H2O_c0 + H2O_e0 + 25 ATP_c0 + Chitin --> 25 ADP_c0 + 25 Phosphate_c0 + 25 H+_c0 + N-Acetyl-D-glucosamine_c0
GPR,
Lower bound,0
Upper bound,1000


In [271]:
## Simulate our model growth, setting the objective to the default of maximizing community biomass

comm.set_objective()

# Add capacity constraints (Var 1- Element uptake limit, Var 2- Kinetic coefficient, Var3- FullThermo constraints)
    #The Kinetic coefficient is a number that is adjustable.
comm.constrain(None,None,None)

# Run FBA -- output is Biomass & Metabolite Uptake and Secretion
output = comm.run(media1)
print(comm.model.summary())

Objective
1.0 bio1 = 0.3099265111158323

Uptake
------
 Metabolite       Reaction      Flux  C-Number  C-Flux
cpd00007_e0 EX_cpd00007_e0     87.73         0   0.00%
cpd00009_e0 EX_cpd00009_e0    0.2518         0   0.00%
cpd00030_e0 EX_cpd00030_e0 0.0008697         0   0.00%
cpd00034_e0 EX_cpd00034_e0 0.0008697         0   0.00%
cpd00048_e0 EX_cpd00048_e0   0.07017         0   0.00%
cpd00058_e0 EX_cpd00058_e0 0.0008697         0   0.00%
cpd00063_e0 EX_cpd00063_e0 0.0008697         0   0.00%
cpd00067_e0 EX_cpd00067_e0     92.32         0   0.00%
cpd00099_e0 EX_cpd00099_e0 0.0008697         0   0.00%
cpd00149_e0 EX_cpd00149_e0 0.0008697         0   0.00%
cpd00205_e0 EX_cpd00205_e0 0.0008697         0   0.00%
cpd00254_e0 EX_cpd00254_e0 0.0008697         0   0.00%
cpd10515_e0 EX_cpd10515_e0  0.003479         0   0.00%
cpd11683_e0 EX_cpd11683_e0        10         8 100.00%
cpd17041_c1    rxn13782_c1    0.2944         0   0.00%
cpd17041_c2    rxn13782_c2    0.0155         0   0.00%
cpd17042_c

In [272]:
flux = comm.solution.fluxes
pd.set_option('display.max_rows', None)
print(flux)

rxn02342_c1            0.000000e+00
rxn08734_c1            0.000000e+00
rxn00199_c1            0.000000e+00
rxn10126_c1            0.000000e+00
rxn05040_c1            0.000000e+00
rxn08556_c1            0.000000e+00
rxn00001_c1            0.000000e+00
rxn02376_c1            0.000000e+00
rxn00568_c1            0.000000e+00
rxn08971_c1            0.000000e+00
rxn00148_c1           -1.404033e+02
rxn08976_c1            0.000000e+00
rxn00781_c1           -2.082618e-01
rxn00097_c1            1.476750e+00
rxn00178_c1            0.000000e+00
rxn09295_c1            0.000000e+00
rxn00549_c1            0.000000e+00
rxn00612_c1           -7.411836e-02
rxn05466_c1           -7.330150e+00
rxn02380_c1            0.000000e+00
rxn08792_c1            0.000000e+00
rxn05145_c1            2.392426e-01
rxn01872_c1            0.000000e+00
rxn00248_c1            1.811521e-01
rxn00499_c1           -6.344554e+00
rxn00392_c1            1.652436e-03
rxn00747_c1            5.749943e-02
rxn01334_c1            0.000

In [19]:
## Compute interactions from FBA solution and reports out to a dataframe

output = comm.compute_interactions()
HTML(output.to_html(render_links=True, escape=False))

    # NOTE: Positive = Secretion, Negative = Uptake

,IDs,Metabolites/Species,Cjaponicus,Ecoli,Environment
0,cpd00067_e0,H+_e0,-90.786249,57.784994,33.001256
1,cpd04097_e0,Pb_e0,0.0,0.0,0
2,cpd00012_e0,PPi_e0,0.0,0.0,0
3,cpd01030_e0,Salicin_e0,0.0,0.0,0
4,cpd00129_e0,L-Proline_e0,-0.23677,0.23677,0
5,cpd00971_e0,Na+_e0,0.0,0.0,0
6,cpd01012_e0,Cd2+_e0,0.0,0.0,0
7,cpd00635_e0,Cbl_e0,0.0,0.0,0
8,cpd00246_e0,Inosine_e0,0.025413,-0.025413,0
9,cpd00276_e0,GLUM_e0,3.46382,-3.46382,0
